In [1]:
import os
import json
# from shapely.geometry import Polygon
from numpy import random
# from shapely.geometry import Point
import pandas as pd

# libraries for UBER API
from uber_rides.session import Session
from uber_rides.client import UberRidesClient

# libraries for take the capture date
from datetime import datetime, timedelta
# libraries for capture data each 4 min
import threading

In [2]:
from uber_rides.session import Session
session = Session(server_token="TZ9aAN7GMzp49djfXoMil2HJ7XxCs0Zwo8EWXd88")

from uber_rides.client import UberRidesClient
client = UberRidesClient(session)

In [3]:
places = [
  { "name": "Centennial Park",
  "location": [33.7603474,-84.3957012]},
  { "name": "Buckhead Bars",
  "location": [33.8439849,-84.3789694]},
  { "name": "Inman Park",
  "location": [33.7613676,-84.3623401]},
  { "name": "Stone Mountain",
  "location": [33.8053189,-84.1477255]},
  { "name": "Six Flags",
  "location": [33.7706408,-84.5537186]},
  { "name": "Statefarm Arena",
 "location": [33.7572891,-84.3963244]},
 { "name": "Zoo Atlanta",
 "location": [33.7327032,-84.3846396]},
 { "name": "Atlanta High Museum",
 "location": [33.7900632,-84.3877407]},
 { "name": "Fox Theater",
 "location": [33.7724591,-84.3879697]},
 { "name": "Virginia Highlands",
 "location": [33.7795027,-84.3757217]},
 { "name": "Shops at Buckhead",
 "location": [33.838031,-84.3821468]},
 { "name": "Emory University",
 "location": [33.7925239,-84.3261929]},
 { "name": "Georgia State University",
 "location": [33.7530724,-84.3874759]},
 { "name": "Spelman College",
 "location": [33.7463641,-84.4144874]},
 { "name": "Edgewood Bars",
 "location": [33.7544814,-84.3745674]},
  {"name": "Hartsfield Jackson Airport",
  "location": [33.6407282,-84.4277001]},
   {"name":"SunTrust Park",
   "location":[33.8908211,-84.4678309]},
   {"name":"Mercedes Benz Stadium",
   "location":[33.7554483,-84.400855]},
   {"name":"Lenox Square Mall",
    "location":[33.8462925,-84.3621419]},
   {"name":"Piedmont Park",
   "location":[33.7850856,-84.373803]}]

total_estimates = []
estimates = {}
#Global Learning Center GA TECH: 33.7762° N, 84.3895° W

for place in places:
    estimates = {}
    response = client.get_price_estimates(
        start_latitude=33.7762,
        start_longitude=-84.3895,
        end_latitude=place["location"][0],
        end_longitude=place["location"][1]
    )

    estimate = response.json.get('prices')
    #print(estimate)
    #estimate10
    estimates["place"] = place["name"]
    estimates["geometry"] = place["location"]
    autotimeinit = datetime.now()
    autotime = autotimeinit.strftime('%H:%M')
    estimates["autotime"] = autotime
    autohour = autotimeinit.strftime('%H')
    autohour = autohour + ":00"
    estimates["time"] = autohour
    estimates["value"] = estimate
    # last_hour_date_time = datetime.now() - timedelta(hours = 1)
    # list_datetime_capture.append(last_hour_date_time.strftime('%Y-%m-%d %H:%M:%S'))
    total_estimates.append(estimates)
total_estimates

[{'place': 'Centennial Park',
  'geometry': [33.7603474, -84.3957012],
  'autotime': '19:58',
  'time': '19:00',
  'value': [{'localized_display_name': 'UberPool',
    'distance': 1.37,
    'display_name': 'UberPool',
    'product_id': 'a9bd012e-768b-479d-8f04-9cecbb955fb0',
    'high_estimate': 10.0,
    'low_estimate': 6.0,
    'duration': 420,
    'estimate': '$6-9',
    'currency_code': 'USD'},
   {'localized_display_name': 'UberX',
    'distance': 1.37,
    'display_name': 'UberX',
    'product_id': '6d898741-0175-4c71-ad5f-93fc66270d6a',
    'high_estimate': 9.0,
    'low_estimate': 7.0,
    'duration': 420,
    'estimate': '$7-9',
    'currency_code': 'USD'},
   {'localized_display_name': 'UberXL',
    'distance': 1.37,
    'display_name': 'UberXL',
    'product_id': '171a12a5-5b08-40cc-bfb0-416b9256147a',
    'high_estimate': 12.0,
    'low_estimate': 9.0,
    'duration': 420,
    'estimate': '$9-12',
    'currency_code': 'USD'},
   {'localized_display_name': 'Select',
    'dis

In [4]:
import json
with open('outfile.json', 'w') as outfile:
    json.dump(total_estimates, outfile)

In [5]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [6]:
# Define our uberPrices table
class UberPricesNew(Base):
    __tablename__ = 'uberPricesNew'
    id = Column(Integer, primary_key=True)
    place = Column(String)
    lat = Column(Integer)
    lon = Column(Integer)
    dist = Column(Integer)
    display_name = Column(String)
    product_id = Column(String)
    duration = Column(Integer)
    estimate = Column(String)
    high_estimate = Column(Integer)
    low_estimate = Column(Integer)
    autotime = Column(String)
    time = Column(String)


In [7]:
# Right now, this table only exists in python and not in the actual database
# Base.metadata.tables

In [8]:
# Create our database engine
engine = create_engine('sqlite:///UberPricesNew.sqlite')

In [9]:
Base.metadata.create_all(engine)

In [10]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

In [11]:
# Note that adding to the session does not update the table. It queues up those queries.
for values in total_estimates:
    for value in values["value"]:
        session.add(UberPricesNew(place=values["place"], lat=values["geometry"][0], lon=values["geometry"][1], 
                               dist=value["distance"], display_name = value["display_name"], product_id = value["product_id"], 
                               duration = value["duration"], estimate = value["estimate"],
                               high_estimate = value["high_estimate"], low_estimate = value["low_estimate"],
                               autotime=values["autotime"], time=values["time"]
                              ))

In [12]:
# commit() flushes whatever remaining changes remain to the database, and commits the transaction.
session.commit()